In [1]:
import pandas as pd
import numpy as np
import folium
import osmnx as ox # installed via pip
import geopy
from itertools import product
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.colors as mcolors
import openrouteservice as ors  # installed via Terminal since the channel isn´t in the navigator
from openrouteservice.directions import directions
import time
import csv
from methods import *
import geopandas as gpd
import random

locator = geopy.Nominatim(user_agent='TUM_Masterthesis_Hohmann')
client = ors.Client(key='5b3ce3597851110001cf62482ac01804d2d6434a8cdd87785bfb541a')  # Specify your personal API key


edges read from csv-file.


In [2]:
def coord_min_to_decimal(coord_str):
    coord_list = coord_str.split(",")
    coord_dec = int(coord_list[0]) + int(coord_list[1])/60 + int(coord_list[2])/3600
    return coord_dec

def set_style(color):
    # function to style data for the displaying of the solution
    return lambda feature: dict(color=color,
                                opacity=0.5,
                                weight=4, )

In [9]:
data_path = r"C:\Users\phili\OneDrive\Uni\Masterarbeit\Data"
file_intersections = data_path + r'\Intersections.csv'
file_customers = data_path + r'\standorte_7.csv'
file_junctions = data_path + r'\junctions.geojson'
file_connections = data_path + r'\connections.csv'
number_customers = 3
new_connections = True
random.seed(23)

df = pd.read_csv(file_intersections, delimiter=";")

df['Longitude'] = df['Longitude'].apply(coord_min_to_decimal)
df['Latitude'] = df['Latitude'].apply(coord_min_to_decimal)

# change dataframes to dict because i know better how to handle those...
intersections = df['ID'].to_list()
lat_list = df['Latitude'].to_list()
lat = {intersections[i]: lat_list[i] for i in range(len(intersections))}
lon_list = df['Longitude'].to_list()
lon ={intersections[i]: lon_list[i] for i in range(len(intersections))}

#create dictionary for the highways that are at the intersection:
highways = dict()
for i in intersections:
    highways_str = df.loc[df['ID'] == i]['Autobahnen'].to_list()
    highways_list = highways_str[0].split(',')
    highways[i] = highways_list
del i, highways_str, highways_list


# find connections
connections = []
for i in df.index:
    text = df['Verbindungen'][i][1:]
    end_points = text.split(',')
    for j in range(len(end_points)):
        end_points[j] = 'AK' + end_points[j]
        connections.append((df['ID'][i], end_points[j]))

# print(connections)
print(f'{int(len(connections))} connections were added.')
# check if all connections are two-sided
for i,j in connections:
    if (j,i) not in connections:
        print(f'Arc {j,i} is missing.')

# add variables for heuristic
E = set()
dist_graph = {i: dict() for i in intersections}
choosen_gitter_points = {i: dict() for i in intersections}


232 connections were added.


In [10]:
# read customers/depot
D = dict()
customer_coords = dict()

with open(file_customers) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    row_count = 0
    for row in csv_reader:
        # first line is header
        # second line is depot
        if row_count == 0:
            headers = row
        elif row_count == 1:
            D[row[0]] = Node(node_id=row[0], x=float(row[3]), y=float(row[2]), category='Depot')
        else:
            customer_coords[row[0]] = Coord(row[3], row[2])
        row_count +=1

print('Customers are read from file.')

# read junctions for exits

gdf = gpd.read_file(file_junctions)
junc_ids = gdf['id'].to_list()
junc_ids = [s.replace('node/', '') for s in junc_ids]

junc_coords_list = gdf['geometry'].to_list()
junc_coords = []
for g in junc_coords_list:
    if g.geom_type == 'Point':
        coord = g.xy
        junc_coords.append((coord[0][0], coord[1][0]))
junc_coords = {junc_ids[i]: Coord(float(junc_coords[i][0]), float(junc_coords[i][1])) for i in range(len(junc_ids))}
del junc_ids, junc_coords_list
print('Junctions are read from file.')

# choose random customers
choosen_customers = random.sample(list(customer_coords.keys()), number_customers)
choosen_customers = {c: customer_coords[c] for c in choosen_customers}
print(choosen_customers)



Customers are read from file.
Junctions are read from file.
{'5054': lon:6.5762593 , lat:50.5441357, '4212': lon:6.99829 , lat:51.16306, '4014': lon:6.7621643 , lat:51.4021407}


In [11]:
# code from 11.9 18:00
# plot the chosen path to see how good it is, only once for testing
gitter_points = {i: [] for i in intersections}
marker_size = 30

loc = ox.geocode('NRW, Germany')
m_test = folium.Map(location=loc, zoom_start=12)

#plot main and all gitter points
for p in intersections:
    #find gitter points
    abweichung = list(range(-3,4,1))
    factor = 0.00014
    gitter = [(factor * a, factor* b) for a in abweichung for b in abweichung]
    gitter_points[p] = [(lat[p] + j[0], lon[p] + j[1]) for j in gitter]

    # plot main point
    marker_text = p[2:]
    marker = folium.Marker(
        location=(lat[p], lon[p]),  # Set the marker's latitude and longitude
        popup=folium.Popup(p, parse_html=True),  # Set the text content inside a popup
        icon=folium.DivIcon(
            icon_size=(30, 30),  # Adjust the size of the circle as needed
            icon_anchor=(15, 15),  # Position the circle's center
            html=f'<div style="font-size: 10pt; text-align: center;">' +
                 f'<svg width="20" height="20">' +
                 f'<circle cx="10" cy="10" r="9" fill="blue" />' +
                 f'<text x="10" y="12" fill="white" text-anchor="middle">{marker_text}</text>' +  # Variable here
                 f'</svg>' +
                 f'</div>'
        ),
    )
    marker.add_to(m_test)

    # plot the gitter
    plot_gitter = False
    if plot_gitter:
        marker_text = 0
        for loc in gitter_points[p]:
            #print(loc)
            marker = folium.Marker(
                location=loc,  # Set the marker's latitude and longitude
                popup=folium.Popup(p, parse_html=True),  # Set the text content inside a popup
                icon=folium.DivIcon(
                    icon_size=(marker_size, marker_size),  # Adjust the size of the circle as needed
                    #icon_anchor=(10, 10),  # Position the circle's center
                    html=f'<div style="font-size: 10pt; text-align: center;">' +
                         f'<svg width="20" height="20">' +
                         f'<circle cx="10" cy="10" r="9" fill="black" />' +
                         f'<text x="10" y="12" fill="white" text-anchor="middle">{marker_text}</text>' +  # Variable here
                         f'</svg>' +
                         f'</div>'
                ),
            )
            marker.add_to(m_test)
            marker_text +=1
print('\n ---- All points are added to the map! ---- \n')


 ---- All points are added to the map! ---- 



In [12]:
# add the connections
if new_connections:
    list_for_writing = []
    for c in connections:
        start = c[0]
        end = c[1]

        # first find the correct autobahn
        connecting_highway = None
        for a in highways[start]:
            if a in highways[end]:
                connecting_highway = a
                break

        coordinates = gitter_points[start] + gitter_points[end]
        coordinates = [(j,i) for i,j in coordinates] # switch von lat,lon to lon, lat


        # no possible way to know which node points in which direction, so have to check all for directions
        dist_matrix = client.distance_matrix(
            locations =coordinates, # lon, lat
            profile='driving-hgv',
            sources= list(range(len(gitter_points[start]))), # start point as sources
            destinations=list(range(len(gitter_points[start]), len(coordinates) )), # only the end points are needed as destinations
            metrics=['duration'],
            units='km'
        )
        dist_matrix_km = np.array(dist_matrix['durations'])
        min_value = np.min(dist_matrix_km)
        ind_start, ind_end = np.where(dist_matrix_km == min_value)
        ind_start = int(ind_start[0])
        ind_end = int(ind_end[0])


        # plot the route
        # ATTENTION: Longitude and latitude are switched compared to plotting
        coordinates = ((gitter_points[start][ind_start][1],gitter_points[start][ind_start][0]), (gitter_points[end][ind_end][1],gitter_points[end][ind_end][0]))
        direction_params = {'coordinates': coordinates,
                            'profile': 'driving-hgv',
                            'format_out': 'geojson',
                            'preference': 'fastest',
                            'geometry': 'true'}
        route = directions(client=client, **direction_params)
        time.sleep(1.6)  # wait for a short time because the server only allows 40 accesses per minute
        distance, duration = route['features'][0]['properties']['summary'].values()
        gj = folium.GeoJson(route, style_function=set_style(list(mcolors.TABLEAU_COLORS.values())[0]))
        gj.add_to(m_test)

        # save variables for writing to file
        list_to_add = [start, end, ind_start, ind_end, distance]
        list_for_writing.append(list_to_add)
        # save variables to continue
        E.add((start, end))
        choosen_gitter_points[start][end] = [ind_start, ind_end]
        dist_graph[start][end] = distance /1000  # change from m to km
        print(f'Connection {start, end} added successfully!')
    del c

    with open(file_connections, 'w') as csv_file:
        csvwriter = csv.writer(csv_file)
        csvwriter.writerows(list_for_writing)

else:
    # a file with the connections exists already
    E = set()
    with open(file_edges, mode='r') as file_temp:

        # reading the CSV file
        csvFile = csv.reader(file_temp)
        # displaying the contents of the CSV file
        for lines in csvFile:
            if len(lines)>0:
                start = lines[0]
                end = lines[1]
                ind_start = lines[2]
                ind_end = lines[3]
                distance = lines[4]

                E.add((start, end))
                dist_graph[start][end] = distance/1000
                choosen_gitter_points[start, end] = {'start': ind_start,
                                                     'end': ind_end}



m_test.save('map_testing_chosen_path.html')

Connection ('AK1', 'AK32') added successfully!
Connection ('AK1', 'AK37') added successfully!
Connection ('AK2', 'AK74') added successfully!
Connection ('AK2', 'AK3') added successfully!
Connection ('AK3', 'AK2') added successfully!
Connection ('AK3', 'AK70') added successfully!
Connection ('AK3', 'AK35') added successfully!
Connection ('AK3', 'AK74') added successfully!
Connection ('AK4', 'AK25') added successfully!
Connection ('AK4', 'AK48') added successfully!
Connection ('AK5', 'AK6') added successfully!
Connection ('AK5', 'AK7') added successfully!
Connection ('AK5', 'AK18') added successfully!
Connection ('AK5', 'AK28') added successfully!
Connection ('AK6', 'AK5') added successfully!
Connection ('AK6', 'AK7') added successfully!
Connection ('AK6', 'AK26') added successfully!
Connection ('AK7', 'AK5') added successfully!
Connection ('AK7', 'AK6') added successfully!
Connection ('AK7', 'AK19') added successfully!
Connection ('AK7', 'AK71') added successfully!
Connection ('AK8', 'A

In [ ]:
### ---cell not needed anymore --- ###
# create a map for all points to find the subpoints
'''
i = 31
loc = [df['Latitude'][i], df['Longitude'][i]]
m = folium.Map(location=loc, zoom_start=21)

marker_text = df['ID'][i]
marker_text = marker_text[2:]
marker_size = 30
marker = folium.Marker(
    location=(df['Latitude'][i], df['Longitude'][i]),  # Set the marker's latitude and longitude
    popup=folium.Popup(df['ID'][i], parse_html=True),  # Set the text content inside a popup
    icon=folium.DivIcon(
        icon_size=(marker_size, marker_size),  # Adjust the size of the circle as needed
        icon_anchor=(marker_size/2, marker_size/2),  # Position the circle's center
        html=f'<div style="font-size: 10pt; text-align: center;">' +
             f'<svg width="{marker_size}" height="{marker_size}">' +
             f'<circle cx="10" cy="10" r="9" fill="blue" />' +
             f'<text x="10" y="12" fill="white" text-anchor="middle">{marker_text}</text>' +  # Variable here
             f'</svg>' +
             f'</div>'
    ),
    tooltip=folium.map.Tooltip("<p><b>{}</b></p><p>{}</p>".format(df['ID'][i],df['Autobahnen'][i]))
)
marker.add_to(m)

abweichung = list(range(-4,5,1))
factor = 0.00012
gitter = [(factor * a, factor* b) for a in abweichung for b in abweichung]
gitter_points = [(df['Latitude'][i] + j[0], df['Longitude'][i] + j[1]) for j in gitter]

for j in range(len(gitter)):
    marker_text = j
    marker = folium.Marker(
        location=(gitter_points[j]),  # Set the marker's latitude and longitude
        popup=folium.Popup(df['ID'][i], parse_html=True),  # Set the text content inside a popup
        icon=folium.DivIcon(
            icon_size=(marker_size, marker_size),  # Adjust the size of the circle as needed
            #icon_anchor=(10, 10),  # Position the circle's center
            html=f'<div style="font-size: 10pt; text-align: center;">' +
                 f'<svg width="20" height="20">' +
                 f'<circle cx="10" cy="10" r="9" fill="black" />' +
                 f'<text x="10" y="12" fill="white" text-anchor="middle">{marker_text}</text>' +  # Variable here
                 f'</svg>' +
                 f'</div>'
        ),
        tooltip=folium.map.Tooltip("<p><b>{}</b></p><p>{}</p>".format(df['ID'][i],df['Autobahnen'][i]))
    )
    marker.add_to(m)

m.save('stupid_map.html')
'''